# Set up

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

print("In Colab:", IN_COLAB)

In Colab: False


In [2]:
if IN_COLAB:
    !pip install transformers datasets torch

In [3]:
if IN_COLAB:
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)

# Imports

In [4]:
import os
import pandas as pd

from datasets import DatasetDict, load_dataset, load_from_disk

import evaluate


import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_scheduler
)

from tqdm.auto import tqdm


# Load data

In [5]:
def generate_dataset(filepath: str, seed = None, validation_split=0.1, test_split=0.1) -> DatasetDict:
    """Load dataset from filepath"""
    ds = load_dataset("csv", data_files=filepath)
    ds_size = len(ds["train"])

    ds = ds.class_encode_column("topic")
    ds = ds.class_encode_column("stance")
    
    # Create a function that converts the `created_at` column to a datetime object
    def convert_to_datetime(example):
        example["created_at"] = pd.to_datetime(example["created_at"])
        return example

    ds = ds.map(convert_to_datetime)

    # Split dataset into train, validation, and test
    ds = ds.shuffle(seed=seed)
    ds_first_split = ds["train"].train_test_split(test_size=test_split)
    ds_second_split = ds_first_split["train"].train_test_split(test_size=validation_split)
    ds = DatasetDict(
        {
            "train": ds_second_split["train"],
            "validation": ds_second_split["test"],
            "test": ds_first_split["test"],
        }
    )

    final_ds_size = sum([len(ds[split]) for split in ds.keys()])
    assert ds_size == final_ds_size, "Dataset size mismatch"

    return ds

In [6]:
path = "UPDATE THIS PATH"
ds = generate_dataset(path)

Using custom data configuration default-0a5351428777341c
Found cached dataset csv (/Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c35ca081c99db987.arrow
Loading cached processed dataset at /Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ca2dd53b6260951e.arrow
Loading cached processed dataset at /Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-06656eb34698d481.arrow
Loading cached processed dataset at /Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3e3313e9846e0349.arrow
Loading cached processed dataset at /Users/bguisard/.cache/huggingface/datasets/csv/default-0a5351428777341c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf7

# Tokenizing the dataset

In [7]:
def dataloaders(ds: DatasetDict, tokenizer, data_collator) -> DatasetDict:
    def tokenize_function(examples):
        return tokenizer(examples["message"], truncation=True)

    tokenized_ds = ds.map(tokenize_function, batched=True)

    # Clean Dataset
    tokenized_ds = tokenized_ds.remove_columns(
        ["message", "created_at", "id", "aggressiveness", "sentiment", "stance"]
    )
    tokenized_ds = tokenized_ds.rename_column("topic", "labels")

    # Create dataloaders
    dls = {}
    for split in tokenized_ds.keys():
        dls[split] = DataLoader(
            tokenized_ds[split],
            shuffle=True,
            batch_size=8,
            collate_fn=data_collator,
        )

    return dls

In [8]:
checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
dls = dataloaders(ds, tokenizer, data_collator)

  0%|          | 0/7 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Fine-tuning the model

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=10, ignore_mismatched_sizes=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(dls['train'])
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2352


In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model.to(device);

In [ ]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in dls['train']:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
model.eval()

# I tried using the `evaluate.combine` method and the `add_batch`, but I was
# getting an an error caused by the multiple classes.

# metrics = evaluate.combine(
#     [
#         evaluate.load("accuracy"),
#         evaluate.load("f1"),
#         evaluate.load("precision"),
#         evaluate.load("recall"),
#         evaluate.load("roc_auc", "multiclass"),
#     ]
# )

labels = []
preds = []
all_preds = []
for batch in dls['validation']:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    softmax = torch.softmax(logits, dim=-1)
    predictions = torch.argmax(softmax, dim=-1)

    labels.extend(batch["labels"].tolist())
    preds.extend(predictions.tolist())
    all_preds.extend(softmax.tolist())


f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
accuracy_metric = evaluate.load("accuracy")
roc_auc_metric = evaluate.load("roc_auc", "multiclass")

# Wrap all the metrics above in a dictionary
metrics = {
    "f1": f1_metric,
    "recall": recall_metric,
    "precision": precision_metric,
    "accuracy": accuracy_metric,
    "roc_auc": roc_auc_metric,
}

# Compute the metrics
results = {}
for metric_name, metric in metrics.items():
    print(metric_name)
    if metric_name == "accuracy":
        r = metric.compute(predictions=preds, references=labels)
    elif metric_name == "roc_auc":
        r = metric.compute(
            prediction_scores=all_preds, references=labels, multi_class="ovr"
        )
    else:
        r = metric.compute(predictions=preds, references=labels, average="weighted")
    results.update(r)

print(results)